In [1]:
#import libraries
import carla
import cv2
import pygame
import numpy as np
import math
from threading import Thread

pygame 2.5.2 (SDL 2.28.3, Python 3.7.16)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [73]:
#define clinet
client = carla.Client('localhost',2000)
world = client.get_world()
bp_library = world.get_blueprint_library()
vehicle_bp_library = world.get_blueprint_library().filter('vehicle')
transform = world.get_map().get_spawn_points()


In [74]:
#spawn main vehicle
start_point = transform[0]
vehicle_bp = vehicle_bp_library.filter('vehicle.bmw.grandtourer')
main_vehicle = world.spawn_actor(vehicle_bp[0],start_point)

In [75]:
#define RGB Camera for main vehicle
camera_bp = bp_library.find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '720')
camera_bp.set_attribute('image_size_y', '460')
camera_transform = carla.Transform(carla.Location(x=-5, z=3))
camera = world.spawn_actor(camera_bp, camera_transform, attach_to=main_vehicle)
def camera_callback(image,data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data),(image.height,image.width,4))

image_w = camera_bp.get_attribute('image_size_x').as_int()
image_h = camera_bp.get_attribute('image_size_y').as_int()

camera_data = {'image': np.zeros((image_h,image_w,4))}
camera.listen(lambda image: camera_callback(image,camera_data))

In [76]:
#control speed for main vehicle
PREFERRED_SPEED = 30 
SPEED_THRESHOLD = 2
def maintain_speed(s):
    if s >= PREFERRED_SPEED:
        return 0
    elif s < PREFERRED_SPEED - SPEED_THRESHOLD:
        return 0.8 
    else:
        return 0.4 

In [77]:
#spawn test vehicle
test_vehicle_bp = vehicle_bp_library.filter('vehicle.mini.cooper_s')
test_vehicle_points = carla.Transform(carla.Location(x=-52.280296, y=46.886234, z=0.067663), carla.Rotation(pitch=-0.004399, yaw=-90.062912, roll=0.0000))
test_vehicle = world.spawn_actor(test_vehicle_bp[0],test_vehicle_points)

In [78]:
# Render object to keep and pass the PyGame surface
class RenderObject(object):
    def __init__(self, width, height):
        init_image = np.random.randint(0,255,(height,width,3),dtype='uint8')
        self.surface = pygame.surfarray.make_surface(init_image.swapaxes(0,1))

# Camera sensor callback, reshapes raw data from camera into 2D RGB and applies to PyGame surface
def pygame_callback(data, obj):
    img = np.reshape(np.copy(data.raw_data), (data.height, data.width, 4))
    img = img[:,:,:3]
    img = img[:, :, ::-1]
    obj.surface = pygame.surfarray.make_surface(img.swapaxes(0,1))

In [79]:
# Control test vehicle with keyboard using pygame
class ControlObject(object):
    def __init__(self, veh):
        self._vehicle = veh
        self._steer = 0
        self._throttle = False
        self._brake = False
        self._steer = None
        self._steer_cache = 0
        self._control = carla.VehicleControl()

    
    def parse_control(self, event):
        if event.type == pygame.KEYDOWN:
            self._vehicle.set_autopilot(False)
            if event.key == pygame.K_UP:
                self._throttle = True
            if event.key == pygame.K_DOWN:
                self._brake = True
            if event.key == pygame.K_RIGHT:
                self._steer = 1
            if event.key == pygame.K_LEFT:
                self._steer = -1
        if event.type == pygame.KEYUP:
            if event.key == pygame.K_UP:
                self._throttle = False
            if event.key == pygame.K_DOWN:
                self._brake = False
                self._control.reverse = False
            if event.key == pygame.K_RIGHT:
                self._steer = None
            if event.key == pygame.K_LEFT:
                self._steer = None

    
    def process_control(self):

        if self._throttle: 
            self._control.hand_brake = False
            self._control.throttle = min(self._control.throttle + 0.01, 1)
            self._control.gear = 1
            self._control.brake = False
        elif not self._brake:
            self._control.throttle = 0.0

        if self._brake:
            # If the down arrow is held down when the car is stationary, switch to reverse
            self._control.hand_brake = False
            if self._vehicle.get_velocity().length() < 0.01 and not self._control.reverse:
                self._control.brake = 0.0
                self._control.gear = 1
                self._control.reverse = True
                self._control.throttle = min(self._control.throttle + 0.1, 1)
            elif self._control.reverse:
                self._control.throttle = min(self._control.throttle + 0.1, 1)
            else:
                self._control.throttle = 0.0
                self._control.brake = min(self._control.brake + 0.3, 1)
        else:
            self._control.brake = 0.0

        if self._steer is not None:
            if self._steer == 1:
                self._steer_cache += 0.03
            if self._steer == -1:
                self._steer_cache -= 0.03
            min(0.7, max(-0.7, self._steer_cache))
            self._control.steer = round(self._steer_cache,1)
        else:
            if self._steer_cache > 0.0:
                self._steer_cache *= 0.2
            if self._steer_cache < 0.0:
                self._steer_cache *= 0.2
            if 0.01 > self._steer_cache > -0.01:
                self._steer_cache = 0.0
            self._control.steer = round(self._steer_cache,1)

        # Ápply the control parameters to the test vehicle
        self._vehicle.apply_control(self._control)

In [80]:
# Initialise the camera for test vehicle
camera_init_trans = carla.Transform(carla.Location(x=-5, z=3), carla.Rotation(pitch=-20))
test_vehicle_cam = world.get_blueprint_library().find('sensor.camera.rgb')
camera = world.spawn_actor(test_vehicle_cam, camera_init_trans, attach_to=test_vehicle)


camera.listen(lambda image: pygame_callback(image, renderObject))


image_w = test_vehicle_cam.get_attribute("image_size_x").as_int()
image_h = test_vehicle_cam.get_attribute("image_size_y").as_int()


renderObject = RenderObject(image_w, image_h)
controlObject = ControlObject(test_vehicle)

In [81]:
#control main vehicle
import sys
sys.path.append('C:\carla\WindowsNoEditor\PythonAPI\carla') #depends on your path
from agents.navigation.global_route_planner import GlobalRoutePlanner

def main_vehicle_control():

    brake_pos = carla.Transform(carla.Location(x=95.991333, y=24.470984, z=0.068305), carla.Rotation(pitch=0.000075, yaw=0.159189, roll=-0.000488))
    point_a = start_point.location

    sampling_resolution = .8
    grp = GlobalRoutePlanner(world.get_map(), sampling_resolution)

    route = grp.trace_route(point_a, brake_pos.location)


    for waypoint in route:
        main_vehicle.set_transform(waypoint[0].transform)
        cv2.imshow('main vehicle',camera_data['image'])
        cv2.waitKey(40)
        world.tick()
        if cv2.waitKey(1) == ord('q'):
            break
        #main_vehicle.apply_control(carla.VehicleControl(throttle=0.0, steer=0.0,hand_brake=True))
        v = main_vehicle.get_velocity()
        speed = round(3.6 * math.sqrt(v.x**2 + v.y**2 + v.z**2),0)
        estimated_throttle = maintain_speed(speed)
        main_vehicle.apply_control(carla.VehicleControl(throttle=estimated_throttle, 
                                                steer=0.0))

        
    cv2.destroyAllWindows()
    camera.stop()
auto_thread = Thread(target=main_vehicle_control)
auto_thread.start()

In [82]:
# Initialise the display
pygame.init()
gameDisplay = pygame.display.set_mode((image_w,image_h), pygame.HWSURFACE | pygame.DOUBLEBUF)

# Game loop
"""def test_vehicle_control():"""
crashed = False

while not crashed:
    world.tick()
    gameDisplay.blit(renderObject.surface, (0,0))
    pygame.display.flip()
    controlObject.process_control()
    for event in pygame.event.get():
            # If the window is closed, break the while loop
        if event.type == pygame.QUIT:
            crashed = True

            # Parse effect of key press event on control state
        controlObject.parse_control(event)
camera.stop()
pygame.quit()
"""keyboard_thread = Thread(target=test_vehicle_control)
keyboard_thread.start()"""


'keyboard_thread = Thread(target=test_vehicle_control)\nkeyboard_thread.start()'

In [31]:
#spawn the pedestrian
pedestrian_bp_library = world.get_blueprint_library().filter('*walker*')
pedestrian_bp = pedestrian_bp_library.filter('walker.pedestrian.0024')
pedestrian_points = carla.Transform(carla.Location(x=58.312057, y=24.831600, z=0.067690), carla.Rotation(pitch=-0.001373, yaw=180.000000, roll=0.000000))
pedestrian = world.spawn_actor(pedestrian_bp[0],pedestrian_points)


In [83]:
#clear all actors
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()
for pedestrian in world.get_actors().filter('*walker*'):
    pedestrian.destroy()
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()

In [72]:
cv2.destroyAllWindows()
camera.stop()
pygame.quit()